## Hackathon PS__1
* Synthetic Data Generation
* The task is to come up with techniques to synthetically generate data   that mimics real-world data. The goal here is to develop a generalisable technique that can be applied to various tabular datasets and achieve high performance on the test data.

In [20]:
try:
    from google.colab import drive
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: using Google CoLab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


* Already have my required installements so directly downloaded from github

In [54]:
CMD = "wget https://raw.githubusercontent.com/Manabjena/hackathon_1/main/Req"

!{CMD}
!pip install -r Req
!pip install tabgan


--2023-03-05 02:42:55--  https://raw.githubusercontent.com/Manabjena/hackathon_1/main/Req
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184 [text/plain]
Saving to: ‘Req.3’

Req.3               100%[===================>]     184  --.-KB/s    in 0s      

2023-03-05 02:42:55 (5.89 MB/s) - ‘Req.3’ saved [184/184]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

* Data Upload (Training- Tabular)

In [23]:
url="https://raw.githubusercontent.com/Manabjena/hackathon_1/main/PS1D_train.csv"
df = pd.read_csv(url,na_values=[' '])
df.head()

,Index,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,3,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,4,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,5,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


* Droping Index Column if present

In [24]:
import io
df1=df
if(df1.columns[0]=='Index'):
  df1=df1.drop(['Index'],axis=1)
df1.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


* Visualizing and checking all Parameters that can be dangerous later (Nan Values)

In [25]:
COLS_USED = df1.columns
COLS_TRAIN = df1.columns[:-1]
df2 = df1[COLS_USED]
df2.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


In [26]:
missing = df2.isnull().sum()
print(missing)

C1        0
C2        0
C3        0
C4        0
C5        0
C6        0
C7        0
C8        0
Target    0
dtype: int64


In [27]:
# Handle missing value if present (dropping is the best option because we dont know what are the columns represnting so doing any statiscal opeartions there would not give a good solution)
df3=df2.dropna()
df3.head()

,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


In [28]:
# Split into training and test sets
df3_x_train, df3_x_test, df3_y_train, df3_y_test = train_test_split(
    df3.drop(df3.columns[-1], axis=1),
    df3[df3.columns[-1]],
    test_size=0.20,
    #shuffle=False,
    random_state=1,
)

In [29]:
# Create dataframe versions for tabular GAN
df3_x_test, df3_y_test = df3_x_test.reset_index(drop=True), \
  df3_y_test.reset_index(drop=True)
df3_y_train = pd.DataFrame(df3_y_train)
df3_y_test = pd.DataFrame(df3_y_test)





# Pandas to Numpy
x_train = df3_x_train.values
x_test = df3_x_test.values
y_train = df3_y_train.values
y_test = df3_y_test.values
x_test

array([[3.    , 0.595 , 0.43  , ..., 0.4525, 0.207 , 0.2725],
       [1.    , 0.66  , 0.51  , ..., 0.7685, 0.3545, 0.3925],
       [2.    , 0.5   , 0.4   , ..., 0.273 , 0.112 , 0.295 ],
       ...,
       [1.    , 0.285 , 0.215 , ..., 0.0415, 0.023 , 0.035 ],
       [3.    , 0.575 , 0.45  , ..., 0.358 , 0.2145, 0.26  ],
       [2.    , 0.505 , 0.38  , ..., 0.391 , 0.1195, 0.1515]])

## Numericals values should be pre processed for a beter output

In [30]:
from sklearn import preprocessing
mm=preprocessing.MinMaxScaler()
x_train=mm.fit_transform(x_train)
x_test=mm.transform(x_test)
x_test

array([[1.        , 0.7398374 , 0.6372549 , ..., 0.36471068, 0.39370829,
        0.37101248],
       [0.        , 0.84552846, 0.79411765, ..., 0.62224939, 0.6749285 ,
        0.53744799],
       [0.5       , 0.58536585, 0.57843137, ..., 0.21841891, 0.21258341,
        0.40221914],
       ...,
       [0.        , 0.23577236, 0.21568627, ..., 0.02974735, 0.042898  ,
        0.04160888],
       [1.        , 0.70731707, 0.67647059, ..., 0.28769356, 0.40800763,
        0.35367545],
       [0.5       , 0.59349593, 0.53921569, ..., 0.31458843, 0.22688275,
        0.20319001]])

### Neural Network Building

In [61]:
# Build the neural network
model = Sequential()
# Hidden 1
model.add(Dense(200, input_dim=x_train.shape[1], activation='relu')) 
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu')) # Hidden 1
model.add(Dense(12, activation='relu'))  # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)   # to protect us from over fitting

## Managed the accuaracy and time limit and hence generated a very nice GAN model

* Model is created to generate the target or output with realtion to features matrix and here no feature details is there so we are not manipualating any rows or columns
* then GAN is fucntioned 

In [62]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=16,callbacks=[monitor], verbose=2,epochs=150)

Epoch 1/150
42/42 - 3s - loss: 53.9256 - val_loss: 13.0665 - 3s/epoch - 60ms/step
Epoch 2/150
42/42 - 0s - loss: 9.2945 - val_loss: 6.3681 - 282ms/epoch - 7ms/step
Epoch 3/150
42/42 - 0s - loss: 7.5996 - val_loss: 6.0599 - 269ms/epoch - 6ms/step
Epoch 4/150
42/42 - 0s - loss: 7.3143 - val_loss: 5.3863 - 263ms/epoch - 6ms/step
Epoch 5/150
42/42 - 0s - loss: 6.6560 - val_loss: 5.1063 - 230ms/epoch - 5ms/step
Epoch 6/150
42/42 - 0s - loss: 6.3222 - val_loss: 4.9117 - 265ms/epoch - 6ms/step
Epoch 7/150
42/42 - 0s - loss: 6.0707 - val_loss: 4.8796 - 262ms/epoch - 6ms/step
Epoch 8/150
42/42 - 0s - loss: 5.9402 - val_loss: 4.9079 - 266ms/epoch - 6ms/step
Epoch 9/150
42/42 - 0s - loss: 6.2879 - val_loss: 4.6172 - 257ms/epoch - 6ms/step
Epoch 10/150
42/42 - 0s - loss: 5.8285 - val_loss: 5.4142 - 267ms/epoch - 6ms/step
Epoch 11/150
42/42 - 0s - loss: 5.6616 - val_loss: 4.5984 - 270ms/epoch - 6ms/step
Epoch 12/150
42/42 - 0s - loss: 5.7484 - val_loss: 4.2751 - 267ms/epoch - 6ms/step
Epoch 13/150


In [63]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred.reshape(1,-1)[0],y_test))
print("Final score (RMSE): {}".format(score))

6/6 [==============================] - 0s 3ms/step
Final score (RMSE): 2.077493094682042


Creating a GAN network to generate artificial data and here we are using a 
small learning rate (0.02)

*  This GAN Generator would prodeuce the data that would be enough good such that the discriminator would not detect .
* this function is a shortcut which can directly shortens the code and would help in both the ways (discriminator and generator)

In [64]:
from tabgan.sampler import GANGenerator

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.01, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(df3_x_train, df3_y_train,\
          df3_x_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

Fitting CTGAN transformers for each column:   0%|          | 0/9 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecat

### Generated_ X_ DATA

In [52]:
gen_x

,C1,C2,C3,C4,C5,C6,C7,C8
0,1,0.290044,0.365444,0.202558,1.391400,1.000215,0.447263,0.275344
1,1,0.360590,0.352599,0.130482,0.337800,0.902102,0.476104,0.181529
2,1,0.422318,0.194685,0.169566,0.795939,0.939120,0.392950,0.441510
3,1,0.399809,0.367493,0.206917,0.789370,1.059881,0.330029,0.434393
4,1,0.371481,0.438562,0.202718,1.184267,0.748254,0.391872,0.308822
...,...,...,...,...,...,...,...,...
1183,2,0.565000,0.480000,0.175000,0.957000,0.388500,0.215000,0.275000
1184,2,0.660000,0.530000,0.170000,1.326000,0.519000,0.262500,0.440000
1185,2,0.655000,0.525000,0.175000,1.348000,0.585500,0.260500,0.394000
1186,2,0.630000,0.500000,0.170000,1.313500,0.559500,0.267000,0.400000


### Generated BY prediction of Target Value

In [53]:
gen_y

0       12
1       -1
2       13
3       12
4        4
        ..
1183    18
1184    13
1185    10
1186    20
1187    12
Name: Target, Length: 1188, dtype: int64

### If we display the results, we can see that the GAN-generated data looks similar to the original. Some values, typically whole numbers in the original data, have fractional values in the synthetic data